In [1]:
!pip install google-generativeai
!pip install python-dotenv



ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\dotenv.exe' -> 'C:\\Python312\\Scripts\\dotenv.exe.deleteme'



In [1]:
import google.generativeai as genai
import json
import re
import os
from dotenv import load_dotenv

load_dotenv()

ci_api_key = os.getenv("CI_API_KEY")

# Configure API key
genai.configure(api_key=ci_api_key)


def generate(text):
    model = genai.GenerativeModel("gemini-2.0-flash")  # Define the model

    # Define input text
    input_text = f"""
    Perform causal inference on the following text:
    
    {text}
    
    The output should be formatted as valid JSON:
    
        "cause": "Cause statement here",
        "effect": "Effect statement here",
        "evidence": "Sentence or phrase where the relationship was found",
        "confidence_score": Confidence score (integer from 1 to 10)
        

    Ensure the JSON output is correctly structured and contains at most 10 relationships.
    """

    # Generate content
    response = model.generate_content(input_text)

    # Combine response text
    response_text = "".join([chunk.text for chunk in response])

    cleaned_text = re.sub(r"```json|```", "", response_text).strip()


    # Convert response to JSON
    try:
        output_json = json.loads(cleaned_text)
    except json.JSONDecodeError:
        output_json = {"error": "Invalid JSON output from model", "raw_response": response_text}

    return output_json  # Return structured output


In [2]:
test_text = "The presidents address to congress tonight stirred the markets. Crypto currencies saw an increase but are now coming to a lull"

result = generate(test_text)
print(json.dumps(result, indent=4))  # Pretty-print the output JSON

[
    {
        "cause": "The president's address to congress",
        "effect": "The markets were stirred.",
        "evidence": "The presidents address to congress tonight stirred the markets.",
        "confidence_score": 9
    },
    {
        "cause": "The president's address to congress",
        "effect": "Crypto currencies saw an increase.",
        "evidence": "The presidents address to congress tonight stirred the markets. Crypto currencies saw an increase",
        "confidence_score": 7
    },
    {
        "cause": "Increase in crypto currencies",
        "effect": "Crypto currencies are now coming to a lull.",
        "evidence": "Crypto currencies saw an increase but are now coming to a lull",
        "confidence_score": 8
    }
]
